# 다중 벡터저장소 검색기(MultiVectorRetriever)

LangChain에서는 문서를 다양한 상황에서 효율적으로 쿼리할 수 있는 특별한 기능, 바로 MultiVectorRetriever를 제공합니다. 이 기능을 사용하면 문서를 여러 벡터로 저장하고 관리할 수 있어, 정보 검색의 정확도와 효율성을 대폭 향상시킬 수 있습니다.

MultiVectorRetriever를 활용해 문서당 여러 벡터를 생성하는 몇 가지 방법을 살펴보겠습니다.

중요한점은 벡터스토어가 여러개 있는게 아니라, `vector_store`에 `parent_docs` 와 `child_docs`를 저장하고, `원본 문서`를 `doc_sotre`에다 저장합니다.

**문서당 여러 벡터 생성 방법 소개**

1. 작은 청크 생성: 문서를 더 작은 단위로 나눈 후, 각 청크에 대해 별도의 임베딩을 생성합니다. 이 방식을 사용하면 문서의 특정 부분에 좀 더 세심한 주의를 기울일 수 있습니다. 이 과정은 ParentDocumentRetriever를 통해 구현할 수 있어, 세부 정보에 대한 탐색이 용이해집니다.

2. 요약 임베딩: 각 문서의 요약을 생성하고, 이 요약으로부터 임베딩을 만듭니다. 이 요약 임베딩은 문서의 핵심 내용을 신속하게 파악하는 데 큰 도움이 됩니다. 문서 전체를 분석하는 대신 핵심적인 요약 부분만을 활용하여 효율성을 극대화할 수 있습니다.

3. 가설 질문 활용: 각 문서에 대해 적합한 가설 질문을 만들고, 이 질문에 기반한 임베딩을 생성합니다. 특정 주제나 내용에 대해 깊이 있는 탐색을 원할 때 이 방법이 유용합니다. 가설 질문은 문서의 내용을 다양한 관점에서 접근하게 해주며, 더 광범위한 이해를 가능하게 합니다.

4. 수동 추가 방식: 사용자가 문서 검색 시 고려해야 할 특정 질문이나 쿼리를 직접 추가할 수 있습니다. 이 방법을 통해 사용자는 검색 과정에서 보다 세밀한 제어를 할 수 있으며, 자신의 요구 사항에 맞춘 맞춤형 검색이 가능해집니다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import PDFPlumberLoader

loader = PDFPlumberLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

In [3]:
print(docs[5].page_content)

1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육
영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언
KEY Contents
n 영국 블레츨리 파크에서 개최된 AI 안전성 정상회의에 참가한 28개국들이 AI 안전 보장을
위한 협력 방안을 담은 블레츨리 선언을 발표
n 첨단 AI를 개발하는 국가와 기업들은 AI 시스템에 대한 안전 테스트 계획에 합의했으며,
영국의 AI 안전 연구소가 전 세계 국가와 협력해 테스트를 주도할 예정
£AI 안전성 정상회의 참가국들, 블레츨리 선언 통해 AI 안전 보장을 위한 협력에 합의
n 2023년 11월 1~2일 영국 블레츨리 파크에서 열린 AI 안전성 정상회의(AI Safety Summit)에
참가한 28개국 대표들이 AI 위험 관리를 위한 ‘블레츨리 선언’을 발표
∙ 선언은 AI 안전 보장을 위해 국가, 국제기구, 기업, 시민사회, 학계를 포함한 모든 이해관계자의 협력이
중요하다고 강조했으며, 특히 최첨단 AI 시스템 개발 기업은 안전 평가를 비롯한 적절한 조치를 취하여
AI 시스템의 안전을 보장할 책임이 있다고 지적
∙ 각국은 AI 안전 보장을 위해 첨단 AI 개발기업의 투명성 향상, 적절한 평가지표와 안전 테스트 도구
개발, 공공부문 역량 구축과 과학 연구개발 등의 분야에서 협력하기로 합의
£영국 총리, 정부 주도의 첨단 AI 시스템 안전 테스트 계획 발표
n 리시 수낙 영국 총리는 AI 안전성 정상회의를 마무리하며 첨단 AI 모델에 대한 안전성 시험 계획
수립과 테스트 수행을 주도할 영국 AI 안전 연구소의 출범을 발표
∙ 첨단 AI 모델의 안전 테스트는 국가 안보와 안전, 사회적 피해를 포함한 여러 잠재적 유해 기능에 대한
시험을 포함하며, 참석자들은 정부 주도의 외부 안전 테스트에 합의
∙ 각국 정부는 테스트와 기타 안전 연구를 위한 공공부문 역량에 투자하고, 테스트 결과가 다른 국가와
관련된 경우 해당 국가와 결과를 공유하며, 적절한 시기에 공동 표준 개발을 위해 노력하기로 합의

# Chunk + 원본문서 검색

대용량 정보를 검색하는 경우, 더 작은 단위로 정보를 임베딩하는 것이 유용할 수 있습니다.

`MultiVectorRetriever` 를 통해 문서를 여러 벡터로 저장하고 관리할 수 있습니다.

`docstore` 에 원본 문서를 저장하고, `vectorstore` 에 임베딩된 문서를 저장합니다.

이로써 문서를 더 작은 단위로 나누어 더 정확한 검색이 가능해집니다. 때에 따라서는 원본 문서의 내용을 조회할 수 있습니다.

In [4]:
import uuid
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers.multi_vector import MultiVectorRetriever

vector_store = Chroma(
    collection_name="small_bigger_chunks",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
)

# 부모 문서의 저장소 계층
store = InMemoryStore()

id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=vector_store,
    byte_store=store,
    id_key=id_key,
)

# 문서 id를 생성
doc_ids = [str(uuid.uuid4()) for _ in docs]

doc_ids

['be350a8f-89fa-4888-95fd-3288ed71683c',
 'fc2c8a8f-59fa-4a68-8a19-1840b8b36a6c',
 '76c8df20-442e-4c42-b6f0-dee1c4670fff',
 'a0bd4e3b-a808-492d-9710-93584173035b',
 'c8f66417-b2b7-409a-b576-833b0baf025b',
 '35440177-8b84-447d-883f-d408f907b83c',
 '8eaf315e-50ea-4af2-8576-a8650e2faad3',
 '978869a3-c2c0-4c6f-9f87-ee6c910ac921',
 'a8e8689f-657c-4415-9217-191bd03aa895',
 '05b5bc67-3514-4499-9f4d-a5dc19b55740',
 '1f0675c3-aba3-4281-9101-92b2337c1cdc',
 '58ebe45c-3f67-4265-8476-e0bed89d664e',
 '02e8f137-e782-44cb-9bd1-f2bf6e4ca6b6',
 '9cf10bc8-80e2-431d-aef5-d912f275411c',
 'f8fb334b-e191-4878-b668-fcb285651c76',
 'd83dabab-b6c6-45f9-a0e3-8c7e591ee1a4',
 'ee027cf6-5e25-4280-8401-58e1c8a16df1',
 '318ad030-17e1-4db2-9c38-af8c260f5c23',
 '44cc6c05-a9f1-47b2-9343-27e93477e514',
 'dd27ee28-565f-4378-ae18-ef5ee28acdb3',
 'e95ebc3d-0dc3-4bd6-bad2-a99ca9773cd1',
 'dc4b666d-f094-493e-85de-d4de3882be03',
 'fb47fbdf-fd8b-4c5c-86bd-465df36e7134']

In [5]:
len(docs)

23

##  parent, child로 split하고

In [6]:
parent_text_splitter = RecursiveCharacterTextSplitter(chunk_size=600)

child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

In [7]:
parent_docs = []

for i, doc in enumerate(docs):
    # 현재 문서의 ID를 가져옵니다.
    _id = doc_ids[i]
    # 현재 문서를 하위 문서로 분할
    parent_doc = parent_text_splitter.split_documents([doc])

    for _doc in parent_doc:
        # metadata에 문서 ID 를 저장
        _doc.metadata[id_key] = _id
    parent_docs.extend(parent_doc)

In [8]:
print(parent_doc)

[Document(metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 22, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4', 'doc_id': 'fb47fbdf-fd8b-4c5c-86bd-465df36e7134'}, page_content='홈페이지 : https://spri.kr/\n보고서와 관련된 문의는 AI정책연구실(jayoo@spri.kr, 031-739-7352)으로 연락주시기 바랍니다.')]


In [9]:
# 생성된 Parent 문서의 메타데이터를 확인합니다.
parent_docs[0].metadata

{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'page': 0,
 'total_pages': 23,
 'Author': 'dj',
 'Creator': 'Hwp 2018 10.0.0.13462',
 'Producer': 'Hancom PDF 1.3.0.542',
 'CreationDate': "D:20231208132838+09'00'",
 'ModDate': "D:20231208132838+09'00'",
 'PDFVersion': '1.4',
 'doc_id': 'be350a8f-89fa-4888-95fd-3288ed71683c'}

In [10]:
child_docs = []
for i, doc in enumerate(docs):
    # 현재 문서의 ID를 가져옵니다.
    _id = doc_ids[i]
    # 현재 문서를 하위 문서로 분할
    child_doc = child_text_splitter.split_documents([doc])
    for _doc in child_doc:
        # metadata에 문서 ID 를 저장
        _doc.metadata[id_key] = _id
    child_docs.extend(child_doc)

In [11]:
# 생성된 Child 문서의 메타데이터를 확인합니다.
child_docs[0].metadata

{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'page': 0,
 'total_pages': 23,
 'Author': 'dj',
 'Creator': 'Hwp 2018 10.0.0.13462',
 'Producer': 'Hancom PDF 1.3.0.542',
 'CreationDate': "D:20231208132838+09'00'",
 'ModDate': "D:20231208132838+09'00'",
 'PDFVersion': '1.4',
 'doc_id': 'be350a8f-89fa-4888-95fd-3288ed71683c'}

In [12]:
print(f"분할된 parent_docs의 개수: {len(parent_docs)}")
print(f"분할된 child_docs의 개수: {len(child_docs)}")

분할된 parent_docs의 개수: 72
분할된 child_docs의 개수: 428


In [13]:
child_doc

[Document(metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 22, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4', 'doc_id': 'fb47fbdf-fd8b-4c5c-86bd-465df36e7134'}, page_content='홈페이지 : https://spri.kr/\n보고서와 관련된 문의는 AI정책연구실(jayoo@spri.kr, 031-739-7352)으로 연락주시기 바랍니다.')]

벡터저장소에 새롭게 생성한 작게 쪼개진 하위문서 집합을 추가합니다.

다음으로는 상위 문서는 생성한 UUID 와 맵핑하여 `docstore` 에 추가합니다.

`mset()` 메서드를 통해 문서 ID와 문서 내용을 key-value 쌍으로 문서 저장소에 저장합니다.

In [15]:
retriever.vectorstore.add_documents(parent_docs)
retriever.vectorstore.add_documents(child_docs)

# docstore 에 원본 문서를 저장
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [16]:
retriever_chunk = retriever.vectorstore.similarity_search("삼성전자가 만든 생성형 ai의 이름은?")

In [17]:
print(f"문서의 검색된 개수", len(retriever_chunk))

문서의 검색된 개수 4


In [18]:
print(retriever_chunk[0].page_content)
print("--------------------------------------------------")
print(retriever_chunk[1].page_content)
print("--------------------------------------------------")
print(retriever_chunk[2].page_content)
print("--------------------------------------------------")
print(retriever_chunk[3].page_content)

☞ 출처 : 삼성전자, ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스’ 공개, 2023.11.08.
삼성전자, ‘삼성 개발자 콘퍼런스 코리아 2023’ 개최, 2023.11.14.
TechRepublic, Samsung Gauss: Samsung Research Reveals Generative AI, 2023.11.08.
10
--------------------------------------------------
☞ 출처 : 삼성전자, ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스’ 공개, 2023.11.08.
삼성전자, ‘삼성 개발자 콘퍼런스 코리아 2023’ 개최, 2023.11.14.
TechRepublic, Samsung Gauss: Samsung Research Reveals Generative AI, 2023.11.08.
10
--------------------------------------------------
▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ···························································10
▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································11
--------------------------------------------------
▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ···························································10
▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································11


- `retriever` 객체의 `search_type` 속성을 `SearchType.mmr`로 설정합니다.
- 이는 검색 시 MMR(Maximal Marginal Relevance) 알고리즘을 사용하도록 지정하는 것입니다.

In [19]:
from langchain.retrievers.multi_vector import SearchType

retriever.search_type = SearchType.mmr
print(retriever.invoke("삼성전자가 만든 생성형ai 이름은?"))

[Document(metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 12, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}, page_content='SPRi AI Brief |\n2023-12월호\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nKEY Contents\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성\nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한\n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\n£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델\n‘삼성 가우스’를 최초 공개\n∙ 정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에\n최적화된 크기의 모델 선택이 가능\n∙ 삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며,\n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n∙ 삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에\n단계적으로 탑재할 계획\nn 삼성 가우스는 △텍스트를 생성하는 언어모

In [20]:
retriever.invoke("안녕?")

[Document(metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 22, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}, page_content='홈페이지 : https://spri.kr/\n보고서와 관련된 문의는 AI정책연구실(jayoo@spri.kr, 031-739-7352)으로 연락주시기 바랍니다.\n'),
 Document(metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 1, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}, page_content='2023년 12월호\nⅠ\n. 인공지능 산업 동향 브리프\n1. 정책/법제\n▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 ·························1\n▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의···························2\n▹ 영국 AI 안전성

In [21]:
from langchain.retrievers.multi_vector import SearchType

# 검색 유형을 similarity_score_threshold로 설정
retriever.search_type = SearchType.similarity_score_threshold
retriever.search_kwargs = {"score_threshold": 0.3}

# 관련 문서 전체를 검색
print(retriever.invoke("삼성전자가 만든 생성형 AI 의 이름은?"))

[Document(metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 12, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}, page_content='SPRi AI Brief |\n2023-12월호\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nKEY Contents\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성\nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한\n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\n£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델\n‘삼성 가우스’를 최초 공개\n∙ 정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에\n최적화된 크기의 모델 선택이 가능\n∙ 삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며,\n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n∙ 삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에\n단계적으로 탑재할 계획\nn 삼성 가우스는 △텍스트를 생성하는 언어모

In [22]:
from langchain.retrievers.multi_vector import SearchType

# 검색 유형을 similarity로 설정, k값을 1로 설정
retriever.search_type = SearchType.similarity
retriever.search_kwargs = {"k": 1}

# 관련 문서 전체를 검색
print(len(retriever.invoke("삼성전자가 만든 생성형 AI 의 이름은?")))

1


# 요약본(Summary)을 벡터스토어에 저장

In [23]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PDFPlumberLoader("./data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)
documents = text_splitter.split_documents(docs)

print(len(documents))

61


In [24]:
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

summary_chain = (
    {"doc": lambda x: x.page_content}
    # 문서 요약을 위한 프롬프트 템플릿 생성
    | ChatPromptTemplate.from_messages(
        [
            ('system','You are an expert in summarizing documents in Korean.'),
            ('user','Summarize the following documents in 3 sentences in bullet points format/ \n\n{doc}'),
        ]
    )
    | ChatOpenAI(model='gpt-4o-mini', temperature=0)
    | StrOutputParser()
)

In [25]:
# 동시다발적으로 빠르게 요약문 생성. batch는 동시에 처리해주는 역할을 한다. max_concurrency은 동시에 10개를 생성. 이게 없으면 순간 한번에 많이 요청하므로 오류가 발생할 수 있다.
summaries = summary_chain.batch(documents, {"max_concurrency": 10})

In [26]:
len(summaries)

61

In [27]:
print(documents[33].page_content)

SPRi AI Brief |
2023-12월호
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
KEY Contents
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델
‘삼성 가우스’를 최초 공개
∙ 정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에
최적화된 크기의 모델 선택이 가능
∙ 삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며,
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙ 삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에


In [28]:
print(summaries[33])

- 삼성전자가 온디바이스에서 작동 가능한 생성 AI 모델 '삼성 가우스'를 공개하였으며, 이는 언어, 코드, 이미지의 3개 모델로 구성되어 있다.
- 삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없으며, 안전한 데이터로 학습되어 다양한 상황에 최적화된 모델 선택이 가능하다.
- 삼성전자는 2023년 11월 8일 '삼성 AI 포럼 2023'에서 삼성 가우스를 처음 소개하고, 이를 다양한 제품에 단계적으로 탑재할 계획이다.


`Chroma` 벡터 저장소를 초기화하여 자식 청크(child chunks)를 인덱싱합니다. 이때 `OpenAIEmbeddings를` 임베딩 함수로 사용합니다.

문서 ID를 나타내는 키로 "doc_id"를 사용합니다.

In [29]:
summary_vector_store = Chroma(
    collection_name = "summaries",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small") 
)

store = InMemoryStore()

id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=summary_vector_store,
    byte_store=store,
    id_key=id_key
)

doc_dis = [str(uuid.uuid4()) for _ in documents]

In [30]:
summary_docs = [
    # 요약된 내용을 페이지 콘텐츠로 하고, 문서 ID를 메타데이터로 포함하는 Document 객체를 생성합니다.
    Document(page_content=s, metadata={id_key: documents[i]})
    for i, s in enumerate(summaries)
]

In [31]:
# 요약본의 문서의 개수
len(summary_docs)

61

In [33]:
retriever.vectorstore.add_documents(
    summary_docs
)  # 요약된 문서를 벡터 저장소에 추가합니다.

# 문서 ID와 문서를 매핑하여 문서 저장소에 저장합니다.
retriever.docstore.mset(list(zip(doc_ids, docs)))

ValueError: Expected metadata value to be a str, int, float or bool, got page_content='12
2023년 월호' metadata={'source': './data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': './data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 0, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'} which is a <class 'langchain_core.documents.base.Document'>

Try filtering complex metadata from the document using langchain_community.vectorstores.utils.filter_complex_metadata.